<h3>VVI</h3>

In [7]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import create_tool_calling_agent,AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,SerpAPIWrapper
from langchain_core.tools import Tool

In [8]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [9]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

<h3>Tavily Search Tool</h3>

In [4]:
searchTavily=TavilySearchResults()

In [52]:
searchTavily.invoke(input="How is the weather in Kolkata?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Kolkata', 'region': 'West Bengal', 'country': 'India', 'lat': 22.5697, 'lon': 88.3697, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1730123499, 'localtime': '2024-10-28 19:21'}, 'current': {'last_updated_epoch': 1730123100, 'last_updated': '2024-10-28 19:15', 'temp_c': 27.1, 'temp_f': 80.8, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.5, 'wind_kph': 10.4, 'wind_degree': 192, 'wind_dir': 'SSW', 'pressure_mb': 1008.0, 'pressure_in': 29.76, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 74, 'cloud': 6, 'feelslike_c': 29.9, 'feelslike_f': 85.8, 'windchill_c': 27.1, 'windchill_f': 80.9, 'heatindex_c': 29.9, 'heatindex_f': 85.8, 'dewpoint_c': 22.2, 'dewpoint_f': 72.0, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 11.8, 'gust_kph': 18.9}}"},
 {'url': 'https://www.hindustantimes.com/cities/kolkata-news/kolkata-weather-to

<h3>Wikipedia Search</h3>

In [6]:
wikipediaSearch=WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

<h3>SerpAPI Search</h3>

In [13]:
serpAPISearch=Tool(
    func=SerpAPIWrapper(search_engine="bing").run,
    description="Search Tool from SerpAPI",
    name="serpAPI_Search")

<h3>Retrievers</h3>

In [14]:
# Loaders
loader=WebBaseLoader(web_path="https://docs.smith.langchain.com/overview")

In [15]:
docs=loader.load()

In [16]:
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='\n\n\n\n\nGet started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Cloud)AdministrationSelf-hostingReferenceQuick startOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its o

In [17]:
# Split Documents
documentsObj=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [18]:
documents=documentsObj.split_documents(documents=docs)

In [19]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content="Skip to main contentGo to API DocsSearc

In [20]:
len(documents)

11

In [21]:
# Use Vector DB to Insert Documents
vector=FAISS.from_documents(documents=documents,embedding=SentenceTransformerEmbeddings())

C:\Users\MSI 1\AppData\Local\Temp\ipykernel_12104\925654796.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vector=FAISS.from_documents(documents=documents,embedding=SentenceTransformerEmbeddings())
C:\Users\MSI 1\AppData\Local\Temp\ipykernel_12104\925654796.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vector=FAISS.from_documents(documents=documents,embedding=SentenceTransformerEmbeddings())


In [22]:
retriever=vector.as_retriever()

In [67]:
retriever.invoke(input="What is Langsmith?")

[Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content="4. Log your first trace‚Äã\nTracing to 

<h3>Create Retriever Tool</h3>

In [23]:
retrieverTool=create_retriever_tool(
    retriever=retriever,
    name="langsmith_search",
    description="Search for information about Langsmith. For any questions about Langsmith, you must use this tool!"
)

In [41]:
# Use Both the Tools - search and retrievertool to use in the downstream
tools=[
    retrieverTool,
    searchTavily,
    serpAPISearch,
    wikipediaSearch]

In [42]:
prompt=hub.pull(owner_repo_commit="hwchase17/openai-functions-agent")

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [43]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [44]:
# The agent is responsible for taking in input and deciding what actions to take
# The agent does not execute those actions, that is done by the agent executor

agent=create_tool_calling_agent(llm=llmGemini,tools=tools,prompt=prompt)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [45]:
agentExecutor=AgentExecutor(tools=tools,agent=agent,verbose=True)

In [29]:
agentExecutor.invoke(input={"input":"hi!"})



> Entering new AgentExecutor chain...
{'name': 'langsmith_search', 'description': 'Search for information about Langsmith. For any questions about Langsmith, you must use this tool!', 'parameters': {'type_': 6, 'description': 'Input to the retriever.', 'properties': {'query': {'type_': 1, 'description': 'query to look up in retriever', 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['query'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'serpAPI_Search', 'description': 'Search Tool from SerpAPI', 'parameters': {'type_': 6, 'properties': {'__arg1': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['__arg1'], 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'wikipedia', 'description': '

{'input': 'hi!', 'output': 'Hello! 👋 How can I help you today? 😊 \n'}

In [75]:
agentExecutor.invoke(input={"input":"How can Langsmith help with Testing?"})



> Entering new AgentExecutor chain...
{'name': 'langsmith_search', 'description': 'Search for information about Langsmith. For any questions about Langsmith, you must use this tool!', 'parameters': {'type_': 6, 'description': 'Input to the retriever.', 'properties': {'query': {'type_': 1, 'description': 'query to look up in retriever', 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['query'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type_': 6, 'description': 'Input for the Tavily tool.', 'properties': {'query': {'type_': 1, 'description': 'search query to look up', 'format_': '', 'nullable': False, 'enum': [], 'max_it

{'input': 'How can Langsmith help with Testing?',
 'output': 'Langsmith helps with testing by providing a platform for evaluating and tracking the performance of your AI models. You can create datasets, define evaluators, and run experiments to measure the accuracy and effectiveness of your models. Langsmith also offers tools for tracing and debugging, making it easier to identify and fix issues in your models. \n'}

In [30]:
agentExecutor.invoke(input={'input':"What is the weather at Kolkata now?"})



> Entering new AgentExecutor chain...
{'name': 'langsmith_search', 'description': 'Search for information about Langsmith. For any questions about Langsmith, you must use this tool!', 'parameters': {'type_': 6, 'description': 'Input to the retriever.', 'properties': {'query': {'type_': 1, 'description': 'query to look up in retriever', 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['query'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'serpAPI_Search', 'description': 'Search Tool from SerpAPI', 'parameters': {'type_': 6, 'properties': {'__arg1': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['__arg1'], 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'wikipedia', 'description': '

{'input': 'What is the weather at Kolkata now?',
 'output': 'The weather in Kolkata, West Bengal, India is clear with periodic clouds. The temperature is 81 degrees Fahrenheit, with 3% precipitation and 88% humidity. The wind speed is 1 mph. \n'}

<h3> Adding in Memory</h3>

In [46]:
messageHistory=ChatMessageHistory()

In [47]:
agentWithChatHistory=RunnableWithMessageHistory(
    runnable=agentExecutor,
    get_session_history=lambda session_id: messageHistory,
    input_message_key="input",
    history_messages_key="chat_history"
)

In [48]:
agentWithChatHistory.invoke(
    input={"input":"hi! My Name is Ritish"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
{'name': 'langsmith_search', 'description': 'Search for information about Langsmith. For any questions about Langsmith, you must use this tool!', 'parameters': {'type_': 6, 'description': 'Input to the retriever.', 'properties': {'query': {'type_': 1, 'description': 'query to look up in retriever', 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['query'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type_': 6, 'description': 'Input for the Tavily tool.', 'properties': {'query': {'type_': 1, 'description': 'search query to look up', 'format_': '', 'nullable': False, 'enum': [], 'max_it

{'input': 'hi! My Name is Ritish',
 'chat_history': [],
 'output': 'Nice to meet you, Ritish! 👋 What can I do for you today? 😊 \n'}

In [49]:
agentWithChatHistory.invoke(
    input={"input":"What's my name?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
{'name': 'langsmith_search', 'description': 'Search for information about Langsmith. For any questions about Langsmith, you must use this tool!', 'parameters': {'type_': 6, 'description': 'Input to the retriever.', 'properties': {'query': {'type_': 1, 'description': 'query to look up in retriever', 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['query'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type_': 6, 'description': 'Input for the Tavily tool.', 'properties': {'query': {'type_': 1, 'description': 'search query to look up', 'format_': '', 'nullable': False, 'enum': [], 'max_it

{'input': "What's my name?",
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ritish! 👋 What can I do for you today? 😊 \n', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Ritish! 😊 \n'}

In [50]:
agentWithChatHistory.invoke(
    input={"input":"What exactly is Langsmith?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
{'name': 'langsmith_search', 'description': 'Search for information about Langsmith. For any questions about Langsmith, you must use this tool!', 'parameters': {'type_': 6, 'description': 'Input to the retriever.', 'properties': {'query': {'type_': 1, 'description': 'query to look up in retriever', 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['query'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type_': 6, 'description': 'Input for the Tavily tool.', 'properties': {'query': {'type_': 1, 'description': 'search query to look up', 'format_': '', 'nullable': False, 'enum': [], 'max_it

{'input': 'What exactly is Langsmith?',
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ritish! 👋 What can I do for you today? 😊 \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish! 😊 \n', additional_kwargs={}, response_metadata={})],
 'output': "Langsmith is a platform for building production-grade LLM applications. It helps you closely monitor and evaluate your application, allowing you to ship quickly and confidently. You don't need to use LangChain - Langsmith works independently! \n"}

In [51]:
agentWithChatHistory.invoke(
    input={"input":"Can India qualify for World Test Championship?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
{'name': 'langsmith_search', 'description': 'Search for information about Langsmith. For any questions about Langsmith, you must use this tool!', 'parameters': {'type_': 6, 'description': 'Input to the retriever.', 'properties': {'query': {'type_': 1, 'description': 'query to look up in retriever', 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['query'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type_': 6, 'description': 'Input for the Tavily tool.', 'properties': {'query': {'type_': 1, 'description': 'search query to look up', 'format_': '', 'nullable': False, 'enum': [], 'max_it

{'input': 'Can India qualify for World Test Championship?',
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ritish! 👋 What can I do for you today? 😊 \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish! 😊 \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What exactly is Langsmith?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Langsmith is a platform for building production-grade LLM applications. It helps you closely monitor and evaluate your application, allowing you to ship quickly and confidently. You don't need to use LangChain - Langsmith works independently! \n", additional_kwargs={}, response_metadata={})],
 'output': "India's chances of qualifying for the World Test Championship final are still possible, but the

In [52]:
agentWithChatHistory.invoke(
    input={"input":"Which party won the election in Jammu and Kashmir?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
{'name': 'langsmith_search', 'description': 'Search for information about Langsmith. For any questions about Langsmith, you must use this tool!', 'parameters': {'type_': 6, 'description': 'Input to the retriever.', 'properties': {'query': {'type_': 1, 'description': 'query to look up in retriever', 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['query'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type_': 6, 'description': 'Input for the Tavily tool.', 'properties': {'query': {'type_': 1, 'description': 'search query to look up', 'format_': '', 'nullable': False, 'enum': [], 'max_it

{'input': 'Which party won the election in Jammu and Kashmir?',
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ritish! 👋 What can I do for you today? 😊 \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish! 😊 \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What exactly is Langsmith?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Langsmith is a platform for building production-grade LLM applications. It helps you closely monitor and evaluate your application, allowing you to ship quickly and confidently. You don't need to use LangChain - Langsmith works independently! \n", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Can India qualify for World Test Championship?', additional_kwargs={}, response_

In [53]:
agentWithChatHistory.invoke(
    input={"input":"What's are the questions that I have asked so far?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
{'name': 'langsmith_search', 'description': 'Search for information about Langsmith. For any questions about Langsmith, you must use this tool!', 'parameters': {'type_': 6, 'description': 'Input to the retriever.', 'properties': {'query': {'type_': 1, 'description': 'query to look up in retriever', 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['query'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type_': 6, 'description': 'Input for the Tavily tool.', 'properties': {'query': {'type_': 1, 'description': 'search query to look up', 'format_': '', 'nullable': False, 'enum': [], 'max_it

{'input': "What's are the questions that I have asked so far?",
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ritish! 👋 What can I do for you today? 😊 \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish! 😊 \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What exactly is Langsmith?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Langsmith is a platform for building production-grade LLM applications. It helps you closely monitor and evaluate your application, allowing you to ship quickly and confidently. You don't need to use LangChain - Langsmith works independently! \n", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Can India qualify for World Test Championship?', additional_kwargs={}, response_

In [54]:
agentWithChatHistory.invoke(
    input={"input":"What's my name?"},
    config={"configurable":{"session_id":"<ritish>"}}
)



> Entering new AgentExecutor chain...
{'name': 'langsmith_search', 'description': 'Search for information about Langsmith. For any questions about Langsmith, you must use this tool!', 'parameters': {'type_': 6, 'description': 'Input to the retriever.', 'properties': {'query': {'type_': 1, 'description': 'query to look up in retriever', 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['query'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type_': 6, 'description': 'Input for the Tavily tool.', 'properties': {'query': {'type_': 1, 'description': 'search query to look up', 'format_': '', 'nullable': False, 'enum': [], 'max_it

{'input': "What's my name?",
 'chat_history': [HumanMessage(content='hi! My Name is Ritish', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ritish! 👋 What can I do for you today? 😊 \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ritish! 😊 \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What exactly is Langsmith?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Langsmith is a platform for building production-grade LLM applications. It helps you closely monitor and evaluate your application, allowing you to ship quickly and confidently. You don't need to use LangChain - Langsmith works independently! \n", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Can India qualify for World Test Championship?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="